In [1]:
from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.image import Image
from azureml.core.model import Model

In [2]:
import azureml.core

In [3]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [4]:
from azureml.core.model import Model
model = Model.register(model_path = "sklearn_regression_model.pkl", # this points to a local file
                       model_name = "sklearn_regression_model.pkl", # this is the name the model is registered as
                       tags = {'area': "diabetes", 'type': "regression"},
                       description = "Ridge regression model to predict diabetes",
                       workspace = ws)

print(model.name, model.description, model.version)

Registering model sklearn_regression_model.pkl
sklearn_regression_model.pkl Ridge regression model to predict diabetes 1


In [5]:
%%writefile score.py
import pickle
import json
import numpy
from sklearn.externals import joblib
from sklearn.linear_model import Ridge
from azureml.core.model import Model

def init():
    global model
    # note here "sklearn_regression_model.pkl" is the name of the model registered under
    # this is a different behavior than before when the code is run locally, even though the code is the same.
    model_path = Model.get_model_path('sklearn_regression_model.pkl')
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)

# note you can pass in multiple rows for scoring
def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = numpy.array(data)
        result = model.predict(data)
        # you can return any data type as long as it is JSON-serializable
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

Writing score.py


In [6]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [7]:

from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "myenv.yml",
                                                  description = "Image with ridge regression model",
                                                  tags = {'area': "diabetes", 'type': "regression"}
                                                 )

image = ContainerImage.create(name = "myimage1",
                              # this is the model object
                              models = [model],
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

Creating image
Running................................................
Succeeded
Image creation operation finished for image myimage1:1, operation "Succeeded"


In [9]:
prov_config = AksCompute.provisioning_configuration()
aks_name = 'my-aks-9'

In [10]:
aks_target = ComputeTarget.create(workspace=ws, name=aks_name, provisioning_configuration=prov_config)

In [11]:
%%time
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

Creating................................................................................................................................................
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
None
Wall time: 13min 4s


In [12]:
aks_config = AksWebservice.deploy_configuration()

In [13]:
%%time
aks_service_name ='aks-service-1'

aks_service = Webservice.deploy_from_image(workspace = ws, 
                                           name = aks_service_name,
                                           image = image,
                                           deployment_config = aks_config,
                                           deployment_target = aks_target)
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

Creating service
Running............
SucceededAKS service creation operation finished, operation "Succeeded"
Healthy
Wall time: 1min 17s


In [14]:
%%time
import json

test_sample = json.dumps({'data': [
    [1,2,3,4,5,6,7,8,9,10], 
    [10,9,8,7,6,5,4,3,2,1]
]})
test_sample = bytes(test_sample,encoding = 'utf8')

prediction = aks_service.run(input_data = test_sample)
print(prediction)

[5215.1981315798685, 3726.995485938578]
Wall time: 631 ms


In [16]:
key1, Key2 = aks_service.get_keys()
key1, Key2

('7lewUTGNF4P6pBmVvzyQJZbWNyKWutzM', 'k239tGQQTnEWWOgXj0rl3J2MpYUNX4UW')

In [20]:
%%time

import requests

import json

test_sample = json.dumps({'data': [
    [1,2,3,4,5,6,7,8,9,10], 
    [10,9,8,7,6,5,4,3,2,1]
]})
test_sample = bytes(test_sample,encoding = 'utf8')

# Don't forget to add key to the HTTP header.
headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + key1}

resp = requests.post(aks_service.scoring_uri, test_sample, headers=headers)


print("prediction:", resp.text)

prediction: [5215.1981315798685, 3726.995485938578]
Wall time: 84.4 ms


In [22]:
%%time
aks_service.delete()

No service with name aks-service-1 found to delete.
Wall time: 183 ms


In [23]:
image.delete()

In [24]:
model.delete()

WebserviceException: Received bad response from Model Management Service:
Response Code: 412
Headers: {'Date': 'Wed, 22 May 2019 23:59:17 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'x-ms-client-request-id': 'ab8ca2a0ec8841438c219577b650767d', 'x-ms-client-session-id': '', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload'}
Content: b'{"code":"PreconditionFailed","statusCode":412,"message":"PreconditionFailed","details":[{"code":"DeletionRequired","message":"The model with id sklearn_regression_model.pkl:1 can not be deleted because it is currently being used in one or more images"}]}'